# Creating the Knowledge Graph

## 0. Install necessary packages

In [58]:
#pip install scipy==1.8.0

     --------------------------------------- 36.9/36.9 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3




[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Import the libraries

In [31]:
# Import important libraries
import pandas as pd
import rdflib
from rdflib import Graph, Namespace, RDF, RDFS, XSD, OWL, Literal
from rdflib.namespace import FOAF
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, URIRef
from datetime import datetime

## 2. Create the panda dataframes from the CSVs

In [32]:
# Load the CSVs
df_judgments = pd.read_csv("ECHR_metadata.csv")
df_edges = pd.read_csv("ECHR_edges.csv")

# Remove unneccesary column
df_judgments = df_judgments.drop(columns=df_judgments.columns[0])

In [33]:
# Show the first 5 entries of the dataframe
df_judgments.head()

,itemid,appno,article,conclusion,docname,doctype,doctypebranch,ecli,importance,judgementdate,languageisocode,originatingbody,violation,nonviolation,extractedappno,scl
0,001-100003,54837/08,6;6-1;13,Violation of Art. 6-1;Violation of Art. 13,CASE OF MATOU AND OTHERS v. GREECE,HEJUD,CHAMBER,ECLI:CE:ECHR:2010:0722JUD005483708,4,22/07/2010 00:00:00,ENG,4.0,6;6-1;13,NaN,NaN,NaN
1,001-100006,12286/08,6;6-1;13,Violation of Art. 6-1;Violation of Art. 13,CASE OF TSOUKALAS v. GREECE,HEJUD,CHAMBER,ECLI:CE:ECHR:2010:0722JUD001228608,4,22/07/2010 00:00:00,ENG,4.0,6;6-1;13,NaN,NaN,NaN
2,001-100009,27023/06,6;6-1,Violation of Art. 6-1,CASE OF SEVGÜL ALTIPARMAK v. TURKEY,HEJUD,CHAMBER,ECLI:CE:ECHR:2010:0720JUD002702306,4,20/07/2010 00:00:00,ENG,5.0,6;6-1,NaN,NaN,NaN
3,001-100011,15121/03;15127/03,NaN,NaN,CASE OF ERDOĞAN AND FIRAT v. TURKEY,HEJUD,CHAMBER,ECLI:CE:ECHR:2010:0720JUD001512103,4,20/07/2010 00:00:00,ENG,6.0,NaN,NaN,NaN,NaN
4,001-100015,12186/08,3;5;5-1;5-4,Violation of Art. 3;Violation of Art. 5-1;Viol...,CASE OF A.A. v. GREECE,HEJUD,CHAMBER,ECLI:CE:ECHR:2010:0722JUD001218608,4,22/07/2010 00:00:00,ENG,4.0,3;5;5-1;5-4,NaN,NaN,NaN


In [34]:
df_edges.head()

,itemid,ecli,references,reference_1,reference_2,reference_3,reference_4,reference_5,reference_6,reference_7,...,reference_91,reference_92,reference_93,reference_94,reference_95,reference_96,reference_97,reference_98,reference_99,reference_100
0,001-100024,ECLI:CE:ECHR:2010:0720JUD000506506,"['ECLI:CE:ECHR:1999:0325JUD003110796', 'ECLI:C...",ECLI:CE:ECHR:1999:0325JUD003110796,ECLI:CE:ECHR:1982:0923JUD000715175,ECLI:CE:ECHR:2000:0105JUD003320296,ECLI:CE:ECHR:2009:0623JUD002765105,ECLI:CE:ECHR:1994:0922JUD001361688,ECLI:CE:ECHR:1986:0708JUD000900680,ECLI:CE:ECHR:2000:1123JUD002570194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001-100032,ECLI:CE:ECHR:2010:0720JUD002542405,['ECLI:CE:ECHR:1998:0805JUD002488194'],ECLI:CE:ECHR:1998:0805JUD002488194,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001-100038,ECLI:CE:ECHR:2010:0720JUD000490006,"['ECLI:CE:ECHR:2005:0204JUD004682799', 'ECLI:C...",ECLI:CE:ECHR:2005:0204JUD004682799,ECLI:CE:ECHR:2006:0608JUD001033704,ECLI:CE:ECHR:2010:0107JUD002440704,ECLI:CE:ECHR:2005:0412JUD003637802,ECLI:CE:ECHR:2000:0711JUD004325898,ECLI:CE:ECHR:2007:0111JUD000194804,ECLI:CE:ECHR:2010:0727JUD005021308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001-100042,ECLI:CE:ECHR:2010:0722JUD001898402,"['ECLI:CE:ECHR:2003:0708JUD003094396', 'ECLI:C...",ECLI:CE:ECHR:2003:0708JUD003094396,ECLI:CE:ECHR:2006:0412JUD006573101,ECLI:CE:ECHR:1998:0327JUD002045892,ECLI:CE:ECHR:2003:0724JUD004001698,ECLI:CE:ECHR:1979:0613JUD000683374,ECLI:CE:ECHR:1999:1221JUD003329096,ECLI:CE:ECHR:1994:0526JUD001696990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001-100050,ECLI:CE:ECHR:2010:0722JUD004937507,"['ECLI:CE:ECHR:1999:0429JUD002564294', 'ECLI:C...",ECLI:CE:ECHR:1999:0429JUD002564294,ECLI:CE:ECHR:1995:0210JUD001517589,ECLI:CE:ECHR:2006:0727JUD003699802,ECLI:CE:ECHR:1999:0728JUD002580394,ECLI:CE:ECHR:2000:0627JUD003097996,ECLI:CE:ECHR:2000:1026JUD003021096,ECLI:CE:ECHR:2003:0225JUD005177299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Reformat the date string from the ECHR_metadata table
def reverse_date_string(s):
    return '-'.join(reversed(s.split('-')))

## 3. Create the graph and add triples

In [38]:
# Define namespaces for the RDF vocabulary used in the KG
DBO = Namespace("http://dbpedia.org/ontology/")
SCH = Namespace("https://schema.org/")
EX = Namespace("http://example.org/legal/")

# Load the KG file
kg_graph = Graph()

# Bind the namespaces to prefixes
kg_graph.bind("dbo", DBO)
kg_graph.bind("schema",SCH)
kg_graph.bind("ex",EX)

for inded, row in df_judgments.iterrows():
    
    # create URI for judgment type
    judgment_uri = URIRef(EX + "Judgment/" + str(row['itemid']))
    
    # reformat date string
    judgment_date = row['judgementdate'].replace("/", "-").split(" ")
    judgmentdate = reverse_date_string(judgment_date[0])
    
    # create list of articles
    articles = str(row['article']).split(";")
    
    # create list of violations
    violations = str(row['violation']).split(";")
    
    # create list of non-violations
    nonviolations = str(row['nonviolation']).split(";")

    # add types to the graph
    kg_graph.add((judgment_uri, RDF.type, EX.judgment))

    # add the predicates to the graph
    kg_graph.add((judgment_uri, EX.ecli, Literal(row['ecli'], datatype=XSD.string)))
    kg_graph.add((judgment_uri, EX.date, Literal(judgmentdate, datatype=XSD.date)))
    kg_graph.add((judgment_uri, EX.name, Literal(row['docname'], datatype=XSD.string)))
    kg_graph.add((judgment_uri, EX.legalbody, Literal(row['doctypebranch'], datatype=XSD.string)))
    kg_graph.add((judgment_uri, EX.importance, Literal(row['importance'], datatype=XSD.integer)))
    
    for article in articles:
        if article != 'nan':
            article_uri = URIRef(EX + "Article/" + str(article))
            kg_graph.add((article_uri, RDF.type, EX.Article))
    
    for violation in violations:
        if violation != 'nan':
            kg_graph.add((judgment_uri, EX.violation, URIRef(EX + "Article/" + violation)))
            
    for nonviolation in nonviolations:
        if nonviolation != 'nan':
            kg_graph.add((judgment_uri, EX.nonviolation, URIRef(EX + "Article/" + nonviolation)))
    

    
limit = 50
for i, row in df_edges.iloc[0:5].iterrows():
    
    judg_uri = URIRef(EX + "Judgment/" + str(row['itemid'])) 
    
    ecli = kg_graph.value(judg_uri, EX.ecli, None)
    
    if row.ecli == str(ecli):
        
        nonclean_refs = str(row['references']).replace('[', '')
        nonclean_refs = nonclean_refs.replace(']', '')
        nonclean_refs = nonclean_refs.replace("'", '')
        references = nonclean_refs.split(',')
        
        for ind, reference in enumerate(references):
            reference = reference.strip()
            
            # retrieve the itemid
            r = df_judgments.loc[(df_judgments['ecli'] == reference), 'itemid'].item()
            
            # create the uri
            ref_uri = URIRef(EX + "Judgment/" + str(r)) 

            if reference != 'nan':
                kg_graph.add((judg_uri, EX.reference, ref_uri))
            if ind == limit:
                break
        
        

## 4. Save the graph

In [39]:
kg_graph.serialize(destination= 'graph.ttl',format='turtle')

<Graph identifier=N5b283633fda541c094fbe923029ade74 (<class 'rdflib.graph.Graph'>)>